# Workspace

Initialize the notebook by running (without editing) the first code blocks. The go to work!

In [2]:
## Do not change this cell, only execute it.

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import numpy as np
import scipy as sp
from scipy.optimize import linprog

In [7]:
def solver(payoff_matrix):
    payoff_matrix = np.array(payoff_matrix)
    row_counts, col_counts = payoff_matrix.shape
# 1. Shift matrix to ensure all values are positive
# This ensures the game value V > 0
    shift = abs(payoff_matrix.min()) + 1
    A_shifted = payoff_matrix + shift
# 2. Solve for Row Player (Maximizer)
# We minimize 1/V = sum(p_i/V). Let x_i = p_i/V
# Objective: min (1 * x1 + 1 * x2 + ...)
    c_row = np.ones(row_counts)
# Constraints: A_shifted.T * x >= 1  =>  -A_shifted.T * x <= -1
    A_ub_row = -A_shifted.T
    b_ub_row = -np.ones(col_counts)
    res_row = linprog(c_row, A_ub=A_ub_row, b_ub=b_ub_row, method='highs')
# Game Value V = 1 / sum(x_i)
    v_shifted = 1 / res_row.fun
    row_strategy = res_row.x * v_shifted
# 3. Solve for Column Player (Minimizer) using the Dual
# Objective: max (1 * y1 + 1 * y2 + ...) => min (-1 * y1 - 1 * y2)
    c_col = -np.ones(col_counts)
# Constraints: A_shifted * y <= 1
    A_ub_col = A_shifted
    b_ub_col = np.ones(row_counts)
    res_col = linprog(c_col, A_ub=A_ub_col, b_ub=b_ub_col, method='highs')
    col_strategy = res_col.x * (1 / abs(res_col.fun))
# Final Game Value
    game_value = v_shifted - shift
    print(f"Game Value: {np.round(game_value,3)}")
    print(f"Rose Strategy: {np.round(row_strategy,3).tolist()}")
    print(f"Colin Strategy: {np.round(col_strategy,3).tolist()}")
    return None

## Start Working Below